In order to make sure that the zones for our zonal statistics are consistent, we need to first verify that the lat/long coordinates of each of the LST files are identical. If they are different, we will need to re-calculate zones for each monthly LST before then calculating the zonal statistics. 

In the code below we download 4 LST images for the last 4 months of 2020 (December, November, October, and September). From what we can tell, the footprint for the lat/long coordinates appears exactly identical. 

In [ ]:
import typing

import geopandas as gpd

# rioxarray is not directly referenced, but its `rio` extension of `xarray` is
import rioxarray
import xarray as xr

In [ ]:
geolocation_gdf = typing.cast(gpd.GeoDataFrame, gpd.read_file("geolocations.geojson"))
geolocation_gdf

In [ ]:
def global_lst_url(*, year: int, month: int) -> str:
    """Return the URL for the global average Land Surface Temperature data file
    for a specific year and month."""

    # Can check status of CEDA core archives at https://stats.uptimerobot.com/vZPgQt7YnO
    # Currently `dap` is down.

    return (
        "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/"
        f"MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/{year}/{month:02d}/"
        f"ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-{year}{month:02d}01000000-fv2.00.nc"
        # Must add `#mode=bytes` to the end.
        # See https://github.com/Unidata/netcdf4-python/issues/1043
        "#mode=bytes"
    )


def lst(*, year: int, month: int) -> xr.Dataset:
    with xr.open_dataset(global_lst_url(year=year, month=month)) as ds:
        return typing.cast(xr.Dataset, ds.squeeze(drop=True).rio.write_crs("EPSG:4326"))

In [ ]:
minx, miny, maxx, maxy = geolocation_gdf.total_bounds
lon = slice(minx, maxx)
lat = slice(miny, maxy)

ds1 = lst(year=2009, month=10).sel(lon=lon, lat=lat)
ds2 = lst(year=2015, month=6).sel(lon=lon, lat=lat)
ds3 = lst(year=2020, month=1).sel(lon=lon, lat=lat)

In [ ]:
print(all(ds1.lon == ds2.lon))
print(all(ds1.lat == ds2.lat))
print(all(ds2.lon == ds3.lon))
print(all(ds2.lat == ds3.lat))